# Section 4: Building a Profit and Loss Dashboard

In this section, you'll learn to:
- Analyze wallet trading performance with PnL metrics
- Track realized and unrealized profits
- View detailed token-level PnL breakdown
- Calculate win rates and trading statistics

## APIs Used in This Section

### REST API Endpoints:
- **`GET /wallet/v2/pnl/summary`** - Get overall PnL summary with trading statistics
- **`GET /wallet/v2/pnl/details`** - Get detailed PnL breakdown per token

Let's build a comprehensive profit and loss tracking system!

## Setup and Imports

**Important**: This section requires a **Business API Key** from Birdeye Data Services.

### Getting Your Business API Key

The wallet PnL features require access to endpoints that are only available with a Business package. You will be provided with a Business API key.

**Add the Business API key to your .env file:**
```
BDS_API_KEY=your_business_api_key_here
```

If you don't have a Business API key, you can:
1. Contact workshop host for a temporary key
2. Sign up for a Business plan at [Birdeye Data Services](https://bds.birdeye.so)
3. Continue with the demo to see how the features work

In [ ]:
# Import required libraries
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from IPython.display import display, HTML, Markdown, clear_output
import ipywidgets as widgets
from datetime import datetime, timedelta
import json
import time
import warnings
warnings.filterwarnings('ignore')

# Force reload of utils module to ensure latest version
import utils
import importlib
importlib.reload(utils)

# Import our custom utilities
from utils import (
    BirdeyeDataServices,
    format_currency,
    check_api_key
)

print("All modules imported successfully!")

# Check Business API key and initialize client
if check_api_key('business'):
    # Initialize Birdeye Data Services client with Business API key
    birdeye = BirdeyeDataServices(api_key_type='business')
    print("Setup complete! Ready to analyze PnL with Business API.")
else:
    print("Please configure BDS_API_KEY in your .env file")
    print("Contact your workshop host for a Business API key")

### 🔧 Troubleshooting Note

**If you encounter import errors:**
1. **Restart the kernel**: Go to `Kernel` → `Restart` in the menu
2. **Re-run the import cell above**
3. **Check your .env file** contains the correct Business API key

All required functions are available in the `utils.py` module.

## Step 1: Configure Wallet Address

Let's set up the wallet address we want to analyze:

In [ ]:
# Example wallet addresses for testing
EXAMPLE_WALLETS = {
    "Example Wallet 1": "5C2NZrHG4pypBoZhq7EroCeeh3G7LTqYh3z3E2SkJeFH",
    "Example Wallet 2": "2EEGkWVGrQet8u6U21tBAxyL9tH2HqJZVZ1EEBN9bZEt",
    "Custom": ""
}

# Create wallet selector
wallet_dropdown = widgets.Dropdown(
    options=list(EXAMPLE_WALLETS.keys()),
    value="Example Wallet 1",
    description='Wallet:',
    style={'description_width': 'initial'}
)

wallet_input = widgets.Text(
    value=EXAMPLE_WALLETS["Example Wallet 1"],
    placeholder='Enter Solana wallet address',
    description='Address:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='600px')
)

def on_wallet_change(change):
    if change['new'] == 'Custom':
        wallet_input.value = ''
    else:
        wallet_input.value = EXAMPLE_WALLETS[change['new']]

wallet_dropdown.observe(on_wallet_change, names='value')

def get_selected_wallet():
    return wallet_input.value.strip()

display(wallet_dropdown)
display(wallet_input)

# Set initial wallet
current_wallet = get_selected_wallet()
print(f"\nCurrent wallet: {current_wallet}")

## Step 2: Get PnL Summary

Let's fetch the overall profit and loss summary for the wallet across different time periods:

In [ ]:
# Duration selector
duration_selector = widgets.Dropdown(
    options=['all', '90d', '30d', '7d', '24h'],
    value='all',
    description='Duration:',
    style={'description_width': 'initial'}
)

fetch_button = widgets.Button(
    description='Fetch PnL Summary',
    button_style='primary',
    icon='chart-line'
)

output_area = widgets.Output()

def fetch_pnl_summary(b):
    with output_area:
        clear_output(wait=True)
        wallet_address = get_selected_wallet()
        
        if not wallet_address:
            print("❌ Please enter a wallet address")
            return
        
        duration = duration_selector.value
        print(f"Fetching PnL summary for wallet: {wallet_address}")
        print(f"Duration: {duration}")
        print("\n" + "="*60)
        
        pnl_data = birdeye.get_wallet_pnl_summary(wallet_address, duration)
        
        if pnl_data and 'data' in pnl_data and 'summary' in pnl_data['data']:
            summary = pnl_data['data']['summary']
            
            # Trading Statistics
            print("\n📊 TRADING STATISTICS")
            print("="*60)
            counts = summary.get('counts', {})
            print(f"Unique Tokens Traded: {summary.get('unique_tokens', 0)}")
            print(f"Total Trades: {counts.get('total_trade', 0)}")
            print(f"  • Buys: {counts.get('total_buy', 0)}")
            print(f"  • Sells: {counts.get('total_sell', 0)}")
            print(f"Wins: {counts.get('total_win', 0)}")
            print(f"Losses: {counts.get('total_loss', 0)}")
            win_rate = counts.get('win_rate', 0) * 100
            print(f"Win Rate: {win_rate:.2f}%")
            
            # Cash Flow
            print("\n💰 CASH FLOW")
            print("="*60)
            cashflow = summary.get('cashflow_usd', {})
            total_invested = cashflow.get('total_invested', 0)
            total_sold = cashflow.get('total_sold', 0)
            print(f"Total Invested: {format_currency(total_invested)}")
            print(f"Total Sold: {format_currency(total_sold)}")
            
            # PnL Metrics
            print("\n📈 PROFIT & LOSS")
            print("="*60)
            pnl = summary.get('pnl', {})
            realized_profit = pnl.get('realized_profit_usd', 0)
            realized_percent = pnl.get('realized_profit_percent', 0) * 100
            unrealized = pnl.get('unrealized_usd', 0)
            total_pnl = pnl.get('total_usd', 0)
            avg_profit = pnl.get('avg_profit_per_trade_usd', 0)
            
            # Color coding for profits/losses
            realized_color = "🟢" if realized_profit >= 0 else "🔴"
            unrealized_color = "🟢" if unrealized >= 0 else "🔴"
            total_color = "🟢" if total_pnl >= 0 else "🔴"
            
            print(f"Realized Profit: {realized_color} {format_currency(realized_profit)} ({realized_percent:+.2f}%)")
            print(f"Unrealized PnL: {unrealized_color} {format_currency(unrealized)}")
            print(f"Total PnL: {total_color} {format_currency(total_pnl)}")
            print(f"Avg Profit per Trade: {format_currency(avg_profit)}")
            
            # Create visualization
            create_pnl_summary_chart(summary)
            
        else:
            print("❌ Could not fetch PnL summary data")
            if pnl_data:
                print(f"Response: {json.dumps(pnl_data, indent=2)}")

def create_pnl_summary_chart(summary):
    """Create visualization for PnL summary"""
    pnl = summary.get('pnl', {})
    
    # Create subplots
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('PnL Breakdown', 'Trading Performance'),
        specs=[[{'type': 'bar'}, {'type': 'indicator'}]]
    )
    
    # PnL Breakdown Bar Chart
    realized = pnl.get('realized_profit_usd', 0)
    unrealized = pnl.get('unrealized_usd', 0)
    
    fig.add_trace(
        go.Bar(
            x=['Realized', 'Unrealized', 'Total'],
            y=[realized, unrealized, realized + unrealized],
            marker_color=['green' if realized >= 0 else 'red',
                         'orange' if unrealized >= 0 else 'darkred',
                         'lightgreen' if (realized + unrealized) >= 0 else 'lightcoral'],
            text=[format_currency(realized), format_currency(unrealized), 
                  format_currency(realized + unrealized)],
            textposition='outside'
        ),
        row=1, col=1
    )
    
    # Win Rate Indicator
    counts = summary.get('counts', {})
    win_rate = counts.get('win_rate', 0) * 100
    
    fig.add_trace(
        go.Indicator(
            mode="gauge+number+delta",
            value=win_rate,
            title={'text': "Win Rate %"},
            delta={'reference': 50},
            gauge={
                'axis': {'range': [0, 100]},
                'bar': {'color': "darkblue"},
                'steps': [
                    {'range': [0, 33], 'color': "lightcoral"},
                    {'range': [33, 66], 'color': "lightyellow"},
                    {'range': [66, 100], 'color': "lightgreen"}],
                'threshold': {
                    'line': {'color': "red", 'width': 4},
                    'thickness': 0.75,
                    'value': 50}
            }
        ),
        row=1, col=2
    )
    
    fig.update_layout(
        height=400,
        showlegend=False,
        title_text="Wallet PnL Summary Dashboard"
    )
    
    fig.show()

fetch_button.on_click(fetch_pnl_summary)

display(duration_selector)
display(fetch_button)
display(output_area)

## Step 3: Get Detailed PnL by Token

Now let's analyze the profit and loss for each individual token:

In [ ]:
# Pagination controls
limit_input = widgets.IntText(
    value=20,
    description='Limit:',
    min=1,
    max=100,
    style={'description_width': 'initial'}
)

offset_input = widgets.IntText(
    value=0,
    description='Offset:',
    min=0,
    style={'description_width': 'initial'}
)

fetch_details_button = widgets.Button(
    description='Fetch Token Details',
    button_style='success',
    icon='list'
)

details_output = widgets.Output()

def fetch_pnl_details(b):
    with details_output:
        clear_output(wait=True)
        wallet_address = get_selected_wallet()
        
        if not wallet_address:
            print("❌ Please enter a wallet address")
            return
        
        limit = limit_input.value
        offset = offset_input.value
        
        print(f"Fetching detailed PnL for wallet: {wallet_address}")
        print(f"Limit: {limit}, Offset: {offset}")
        print("\n" + "="*80)
        
        pnl_details = birdeye.get_wallet_pnl_details(wallet_address, limit, offset)
        
        if pnl_details and 'data' in pnl_details:
            data = pnl_details['data']
            
            # Display metadata
            if 'meta' in data:
                meta = data['meta']
                print("\n📋 METADATA")
                print("="*80)
                print(f"Wallet: {meta.get('address', 'N/A')}")
                print(f"Currency: {meta.get('currency', 'usd').upper()}")
                print(f"Timestamp: {meta.get('time', 'N/A')}")
            
            # Display summary
            if 'summary' in data:
                summary = data['summary']
                print("\n📊 OVERALL SUMMARY")
                print("="*80)
                counts = summary.get('counts', {})
                cashflow = summary.get('cashflow_usd', {})
                pnl = summary.get('pnl', {})
                
                print(f"Unique Tokens: {summary.get('unique_tokens', 0)}")
                print(f"Total Trades: {counts.get('total_trade', 0)} (Buys: {counts.get('total_buy', 0)}, Sells: {counts.get('total_sell', 0)})")
                print(f"Win Rate: {counts.get('win_rate', 0) * 100:.2f}%")
                print(f"Total Invested: {format_currency(cashflow.get('total_invested', 0))}")
                print(f"Total Sold: {format_currency(cashflow.get('total_sold', 0))}")
                print(f"Current Value: {format_currency(cashflow.get('current_value', 0))}")
                print(f"Total PnL: {format_currency(pnl.get('total_usd', 0))}")
            
            # Display token details
            if 'tokens' in data and data['tokens']:
                tokens = data['tokens']
                print(f"\n\n🪙 TOKEN DETAILS ({len(tokens)} tokens)")
                print("="*80)
                
                # Create DataFrame for better display
                token_data = []
                for token in tokens:
                    pnl = token.get('pnl', {})
                    counts = token.get('counts', {})
                    cashflow = token.get('cashflow_usd', {})
                    
                    token_data.append({
                        'Symbol': token.get('symbol', 'N/A'),
                        'Address': token.get('address', 'N/A')[:8] + '...',
                        'Trades': counts.get('total_trade', 0),
                        'Invested': cashflow.get('total_invested', 0),
                        'Sold': cashflow.get('total_sold', 0),
                        'Current Value': cashflow.get('current_value', 0),
                        'Realized PnL': pnl.get('realized_profit_usd', 0),
                        'Realized %': pnl.get('realized_profit_percent', 0) * 100,
                        'Unrealized PnL': pnl.get('unrealized_usd', 0),
                        'Total PnL': pnl.get('total_usd', 0),
                        'Total %': pnl.get('total_percent', 0) * 100
                    })
                
                df = pd.DataFrame(token_data)
                
                # Format currency columns
                currency_cols = ['Invested', 'Sold', 'Current Value', 'Realized PnL', 'Unrealized PnL', 'Total PnL']
                for col in currency_cols:
                    df[col] = df[col].apply(lambda x: f"${x:,.2f}")
                
                # Format percentage columns
                df['Realized %'] = df['Realized %'].apply(lambda x: f"{x:+.2f}%")
                df['Total %'] = df['Total %'].apply(lambda x: f"{x:+.2f}%")
                
                # Display styled DataFrame
                display(df)
                
                # Create visualization
                create_token_pnl_chart(tokens)
                
            else:
                print("\n⚠️ No token details available")
                
        else:
            print("❌ Could not fetch PnL details")
            if pnl_details:
                print(f"Response: {json.dumps(pnl_details, indent=2)}")

def create_token_pnl_chart(tokens):
    """Create visualization for token-level PnL"""
    if not tokens:
        return
    
    # Prepare data
    symbols = []
    realized_pnl = []
    unrealized_pnl = []
    total_pnl = []
    
    for token in tokens[:10]:  # Top 10 tokens
        symbols.append(token.get('symbol', 'Unknown'))
        pnl = token.get('pnl', {})
        realized_pnl.append(pnl.get('realized_profit_usd', 0))
        unrealized_pnl.append(pnl.get('unrealized_usd', 0))
        total_pnl.append(pnl.get('total_usd', 0))
    
    # Create grouped bar chart
    fig = go.Figure()
    
    fig.add_trace(go.Bar(
        name='Realized PnL',
        x=symbols,
        y=realized_pnl,
        marker_color='lightblue'
    ))
    
    fig.add_trace(go.Bar(
        name='Unrealized PnL',
        x=symbols,
        y=unrealized_pnl,
        marker_color='orange'
    ))
    
    fig.add_trace(go.Bar(
        name='Total PnL',
        x=symbols,
        y=total_pnl,
        marker_color='green'
    ))
    
    fig.update_layout(
        title='Top 10 Tokens by PnL',
        xaxis_title='Token',
        yaxis_title='PnL (USD)',
        barmode='group',
        height=500,
        hovermode='x unified'
    )
    
    fig.show()

fetch_details_button.on_click(fetch_pnl_details)

print("Configure pagination settings:")
display(widgets.HBox([limit_input, offset_input]))
display(fetch_details_button)
display(details_output)

## Step 4: Complete PnL Dashboard

Let's create an all-in-one interactive dashboard that combines both summary and detailed views:

In [ ]:
# Create comprehensive dashboard
dashboard_duration = widgets.Dropdown(
    options=['all', '90d', '30d', '7d', '24h'],
    value='30d',
    description='Time Period:',
    style={'description_width': 'initial'}
)

dashboard_limit = widgets.IntText(
    value=10,
    description='Top Tokens:',
    min=5,
    max=50,
    style={'description_width': 'initial'}
)

refresh_dashboard_button = widgets.Button(
    description='🔄 Refresh Dashboard',
    button_style='info',
    icon='refresh',
    layout=widgets.Layout(width='200px')
)

dashboard_output = widgets.Output()

def refresh_dashboard(b):
    with dashboard_output:
        clear_output(wait=True)
        wallet_address = get_selected_wallet()
        
        if not wallet_address:
            print("❌ Please enter a wallet address")
            return
        
        duration = dashboard_duration.value
        limit = dashboard_limit.value
        
        print("🎯 COMPREHENSIVE PnL DASHBOARD")
        print("="*80)
        print(f"Wallet: {wallet_address}")
        print(f"Period: {duration}")
        print(f"Analyzing top {limit} tokens...\n")
        
        # Fetch both summary and details
        print("Fetching data...")
        summary_data = birdeye.get_wallet_pnl_summary(wallet_address, duration)
        time.sleep(0.5)  # Rate limiting
        details_data = birdeye.get_wallet_pnl_details(wallet_address, limit, 0)
        
        if summary_data and 'data' in summary_data and 'summary' in summary_data['data']:
            summary = summary_data['data']['summary']
            
            # Key Metrics Display
            print("\n" + "="*80)
            print("📊 KEY PERFORMANCE METRICS")
            print("="*80)
            
            counts = summary.get('counts', {})
            pnl = summary.get('pnl', {})
            cashflow = summary.get('cashflow_usd', {})
            
            # Create metrics display
            total_pnl = pnl.get('total_usd', 0)
            win_rate = counts.get('win_rate', 0) * 100
            total_trades = counts.get('total_trade', 0)
            
            metrics_html = f"""
            <div style='background-color: #f0f0f0; padding: 20px; border-radius: 10px;'>
                <h3 style='margin-top: 0;'>Performance Overview</h3>
                <table style='width: 100%; border-collapse: collapse;'>
                    <tr>
                        <td style='padding: 10px; border-bottom: 1px solid #ddd;'><b>Total PnL:</b></td>
                        <td style='padding: 10px; border-bottom: 1px solid #ddd; color: {'green' if total_pnl >= 0 else 'red'};'>
                            <b>{format_currency(total_pnl)}</b>
                        </td>
                    </tr>
                    <tr>
                        <td style='padding: 10px; border-bottom: 1px solid #ddd;'><b>Win Rate:</b></td>
                        <td style='padding: 10px; border-bottom: 1px solid #ddd;'><b>{win_rate:.2f}%</b></td>
                    </tr>
                    <tr>
                        <td style='padding: 10px; border-bottom: 1px solid #ddd;'><b>Total Trades:</b></td>
                        <td style='padding: 10px; border-bottom: 1px solid #ddd;'><b>{total_trades}</b></td>
                    </tr>
                    <tr>
                        <td style='padding: 10px; border-bottom: 1px solid #ddd;'><b>Unique Tokens:</b></td>
                        <td style='padding: 10px; border-bottom: 1px solid #ddd;'><b>{summary.get('unique_tokens', 0)}</b></td>
                    </tr>
                    <tr>
                        <td style='padding: 10px;'><b>Avg Profit/Trade:</b></td>
                        <td style='padding: 10px;'><b>{format_currency(pnl.get('avg_profit_per_trade_usd', 0))}</b></td>
                    </tr>
                </table>
            </div>
            """
            display(HTML(metrics_html))
            
            # Create comprehensive visualization
            if details_data and 'data' in details_data and 'tokens' in details_data['data']:
                tokens = details_data['data']['tokens']
                
                # Create multi-panel dashboard
                fig = make_subplots(
                    rows=2, cols=2,
                    subplot_titles=(
                        'PnL by Token',
                        'Win Rate Gauge',
                        'Realized vs Unrealized',
                        'Top Performers'
                    ),
                    specs=[
                        [{'type': 'bar'}, {'type': 'indicator'}],
                        [{'type': 'bar'}, {'type': 'bar'}]
                    ],
                    vertical_spacing=0.15,
                    horizontal_spacing=0.1
                )
                
                # Panel 1: Total PnL by Token
                symbols = [t.get('symbol', 'Unknown') for t in tokens[:10]]
                total_pnls = [t.get('pnl', {}).get('total_usd', 0) for t in tokens[:10]]
                colors = ['green' if p >= 0 else 'red' for p in total_pnls]
                
                fig.add_trace(
                    go.Bar(x=symbols, y=total_pnls, marker_color=colors, name='Total PnL'),
                    row=1, col=1
                )
                
                # Panel 2: Win Rate Gauge
                fig.add_trace(
                    go.Indicator(
                        mode="gauge+number",
                        value=win_rate,
                        title={'text': "Win Rate %"},
                        gauge={
                            'axis': {'range': [0, 100]},
                            'bar': {'color': "darkblue"},
                            'steps': [
                                {'range': [0, 33], 'color': "lightcoral"},
                                {'range': [33, 66], 'color': "lightyellow"},
                                {'range': [66, 100], 'color': "lightgreen"}]
                        }
                    ),
                    row=1, col=2
                )
                
                # Panel 3: Realized vs Unrealized
                realized = [t.get('pnl', {}).get('realized_profit_usd', 0) for t in tokens[:5]]
                unrealized = [t.get('pnl', {}).get('unrealized_usd', 0) for t in tokens[:5]]
                top_symbols = [t.get('symbol', 'Unknown') for t in tokens[:5]]
                
                fig.add_trace(
                    go.Bar(x=top_symbols, y=realized, name='Realized', marker_color='lightblue'),
                    row=2, col=1
                )
                fig.add_trace(
                    go.Bar(x=top_symbols, y=unrealized, name='Unrealized', marker_color='orange'),
                    row=2, col=1
                )
                
                # Panel 4: Top Performers by %
                sorted_tokens = sorted(tokens, key=lambda x: x.get('pnl', {}).get('total_percent', 0), reverse=True)[:5]
                perf_symbols = [t.get('symbol', 'Unknown') for t in sorted_tokens]
                perf_percent = [t.get('pnl', {}).get('total_percent', 0) * 100 for t in sorted_tokens]
                perf_colors = ['green' if p >= 0 else 'red' for p in perf_percent]
                
                fig.add_trace(
                    go.Bar(x=perf_symbols, y=perf_percent, marker_color=perf_colors, name='% Return'),
                    row=2, col=2
                )
                
                fig.update_layout(
                    height=800,
                    showlegend=True,
                    title_text=f"Complete PnL Dashboard - {duration.upper()}"
                )
                
                fig.update_xaxes(title_text="Token", row=1, col=1)
                fig.update_yaxes(title_text="PnL (USD)", row=1, col=1)
                fig.update_xaxes(title_text="Token", row=2, col=1)
                fig.update_yaxes(title_text="PnL (USD)", row=2, col=1)
                fig.update_xaxes(title_text="Token", row=2, col=2)
                fig.update_yaxes(title_text="Return %", row=2, col=2)
                
                fig.show()
                
        else:
            print("❌ Could not fetch dashboard data")

refresh_dashboard_button.on_click(refresh_dashboard)

print("\n🎯 COMPLETE PnL DASHBOARD")
print("Configure your dashboard settings and click refresh:")
display(widgets.HBox([dashboard_duration, dashboard_limit]))
display(refresh_dashboard_button)
display(dashboard_output)

## 🎉 Congratulations!

You've successfully built a comprehensive Profit and Loss dashboard! Here's what you've learned:

### Key Takeaways:

1. **PnL Summary API** - Track overall trading performance with:
   - Realized and unrealized profits
   - Win rates and trading statistics
   - Cash flow analysis

2. **PnL Details API** - Analyze token-level performance:
   - Individual token profits/losses
   - Trading history per token
   - Pagination for large portfolios

3. **Data Visualization** - Create professional dashboards:
   - Multi-panel layouts
   - Interactive charts with Plotly
   - Color-coded performance indicators

### Next Steps:

- **Combine with Portfolio Data**: Integrate PnL metrics with net worth tracking
- **Historical Analysis**: Track PnL changes over different time periods
- **Risk Metrics**: Calculate Sharpe ratio, max drawdown, etc.
- **Automated Alerts**: Set up notifications for significant PnL changes

### 📚 Additional Resources:

- [Birdeye Data Services Documentation](https://docs.birdeye.so)
- [Wallet API Reference](https://docs.birdeye.so/reference/wallet-apis)
- [PnL API Guide](https://docs.birdeye.so/reference/get_wallet-v2-pnl-summary)

### 💡 Pro Tips:

1. Use different duration parameters to analyze short-term vs long-term performance
2. Combine PnL data with token price charts for deeper insights
3. Export data to CSV for further analysis in Excel or other tools
4. Set up regular monitoring to track portfolio performance over time

Happy trading and analyzing! 🚀